In [1]:
import numpy as np

# <center> TP2 - Chiffrement affine </center>
<center> 2025/2026 - L. Naert, S. Bouchelaghem, T. Godin </center>


_Certains exemples et textes de ce TP sont tirés de Exercices et Problèmes de cryptographie de Damien Vergnaud, 3ème édition_

__Usage de l'IA générative : interdit__

Dans ce TP, nous étudierons un nouveau type de chiffrement par substitution monoalphabétique : le chiffrement affine.

Pour ce chiffrement, nous développerons des fonctions pour:
1. chiffrer un message en clair à l'aide d'une clef
2. déchiffrer un message (en connaissant la clef)
3. décrypter un message (ne connaissant pas la clef) en faisant appel à l'analyse de fréquence


Par convention, nous appellerons :
- $k$ : la clef
- $E_k$ : la fonction de chiffrement
- $D_k$ : la fonction de déchiffrement
- $m$ : le message en clair
- $m_i$ : la lettre de rang $i$ du message en clair
- $c$ : le message chiffré
- $c_i$ : la lettre de rang $i$ du message chiffré

_Note_ : à tout moment, vous pouvez utiliser des fonctions définies dans les TPs précédents.

In [2]:
# Quelques fonctions utiles (ou pas)
def lettreToEntier(lettre, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    return alphabet.find(lettre)
def entierToLettre(a, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    return alphabet[a]

print("Lettre -> entier : w ->", lettreToEntier('w'))
print("Entier -> Lettre : 22 ->", entierToLettre(22))

Lettre -> entier : w -> 22
Entier -> Lettre : 22 -> w


## Chiffrement Affine

Le chiffrement affine est un chiffrement par substitution monoalphabétique. Ici, la clef de chiffrement $k$ est composée d'un couple d'entiers $(a,b)$ avec $a \in (\mathbb{Z}/n\mathbb{Z})^*$ (l'ensemble des éléments inversibles de $\mathbb{Z}/n\mathbb{Z}$) et $b \in \mathbb{Z}/n\mathbb{Z}$

La fonction de chiffrement correspondante est : 

\begin{align*}
  E_k \colon \mathbb{Z}/26\mathbb{Z} &\to \mathbb{Z}/26\mathbb{Z}\\
  m_i & \mapsto c_i = am_i + b
\end{align*}

Ainsi, si k = (3,4), la lettre codée 6 (donc 'g') sera chiffrée avec la lettre codée $(3*6+4)\mod 26 = 22$ (donc 'w').

### a) Chiffrement/Déchiffrement

> __Question 1 (chiffrement)__ : Définir une fonction `chiffrementAffine(msgClair, a, b, alphabet)` qui étant donné un message en clair `msgClair`, une clef de chiffrement `clef`$=[a,b]$ et un alphabet `alphabet` (par défaut, l'alphabet français) renvoie le message chiffré correspondant.

In [3]:
def chiffrementAffine(msgClair, clef, alphabet = "abcdefghijklmnopqrstuvwxyz") :
    msgCrypt = ""
    for letter in msgClair : 
        letter = lettreToEntier(letter)
        letterCrypt = clef[0] * letter + clef[1]
        letterCrypt = letterCrypt % len(alphabet)
        msgCrypt += entierToLettre(letterCrypt)
    return msgCrypt


try:
    assert chiffrementAffine("messageenclair",[3,4]) == "oqggewqqrklecd"
    assert chiffrementAffine("messageenclair",[23,6]) == "wueegouutazgih"
    print("chiffrementAffine : OK")
except:
    print("chiffrementAffine : ERREUR")


chiffrementAffine : OK


Partant du message chiffré et connaissant la clef de chiffrement (les valeurs de a et b), il est possible de déchiffrer le message.

La fonction de déchiffrement peut s'écrire : 

\begin{align*}
  D_k \colon \mathbb{Z}/26\mathbb{Z} &\to \mathbb{Z}/26\mathbb{Z}\\
  c_i &\mapsto m_i = \alpha c_i + \beta
\end{align*}

avec $\alpha = a^{-1} \mod n$ et $\beta = (-a^{-1}b) \mod n$

où $a^{-1}$ désigne l'inverse modulaire de $a$.

Avant de faire la fonction permettant de déchiffrer un message chiffré connaissant sa clef, il est donc nécessaire de faire une parenthèse plus mathématique sur les inverses modulaires ! 

#### Inversibilité dans $\mathbb{Z}/n\mathbb{Z}$

S'il existe un entier $b$ de $\mathbb{Z}/n\mathbb{Z}$ tel que $ab \mod n = 1$ alors b est appelé __inverse__ de a dans $\mathbb{Z}/n\mathbb{Z}$ et est noté $a^{-1}$.

$a$ est inversible dans $\mathbb{Z}/n\mathbb{Z}$ ssi $a$ et $n$ sont premiers entre eux, c'est à dire qu'ils n'ont aucun diviseur commun (i.e. $pgcd(a,n) = 1$).

> __Question 2 :__
- Faire une fonction `estInversible(a, n)` qui renvoie `True` si $a$ est inversible sur $\mathbb{Z}/n\mathbb{Z}$ et `False`sinon.
- Faire une fonction `listeInversibles(n)` qui renvoie la liste de tous les éléments inversibles de $\mathbb{Z}/n\mathbb{Z}$



In [4]:
def estInversible(a,n):
    #algo d'Euclide
    while n != 0 : 
        rest = a%n
        a = n
        n = rest 
    return a == 1


try:
    assert estInversible(0,180) == False
    assert estInversible(1,180) == True
    assert estInversible(4,180) == False
    assert estInversible(5,180) == False
    assert estInversible(179,180) == True
    assert estInversible(11,180) == True
    assert estInversible(131,180) == True
    assert estInversible(13,26) == False
    assert estInversible(4,26) == False
    assert estInversible(5,26) == True
    print("estInversible : OK")
except:
    print("estInversible : ERREUR")


estInversible : OK


In [5]:
def listeInversibles(n):
    list = []
    for i in range(1,n) : 
        if(estInversible(i,n)) : 
            list.append(i)
    return list

    
print(listeInversibles(14))
try:
    assert listeInversibles(26) == [1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25]
    assert listeInversibles(5) == [1, 2, 3, 4]
    assert listeInversibles(6) == [1, 5]
    print("listeInversibles : OK")
except:
    print("listeInversibles : ERREUR")

    

[1, 3, 5, 9, 11, 13]
listeInversibles : OK


Maintenant que nous savons si un nombre est inversible dans $\mathbb{Z}/n\mathbb{Z}$, il nous faut pouvoir calculer son inverse. Une méthode consiste à trouver les __coefficients de Bezout__ entre $a$ et $n$.

De manière générale, les coefficients de Bezout de $x$ et $y$ sont les entiers relatifs $s$ et $t$ tels que $sx + ty = pgcd(x,y) $ 

Si $a$ est inversible, il s'agit donc de trouver les entiers relatifs $s$ et $t$ tels que $sa + tn = 1 $ car a et n sont premiers entre eux (donc $pgcd(a,n) = 1$)

Comme $tn = 0$ dans $\mathbb{Z}/n\mathbb{Z}$ ($tn$ est un multiple de n), on a $sa=1$ donc $a^{-1} = s\mod n$ (par définition de l'inverse).

>__Question 3 :__
A partir de la fonction `coeffBezout(a, b)` qui renvoie les valeurs de $pgcd(a,b)$, $s$ et $t$ dans une liste, faire une fonction `inverse(a, n)` qui teste si $a$ est inversible et, si oui, renvoie sont inverse. Sinon, renvoie -1.

In [6]:
# sources : cette [vidéo](https://www.youtube.com/watch?v=7o79t2KAKxE&list=PLE8WtfrsTAinMMyQkK_CzXhXU_LHRNXy_&index=3) 
# et [celle-ci](https://www.youtube.com/watch?v=BkK1_FspgYQ).
def coeffBezout(a, b) :
    l1 = np.array([a, 1, 0])
    l2 = np.array([b, 0, 1])
    
    while( l2[0] != 0):
        mult = l1[0]//l2[0]
        ltmp = l2
        l2 = l1 - mult*l2
        l1 = ltmp
    if(l1[0]<0):
        l1 = -l1
        
    return l1.tolist()

try:
    assert coeffBezout(180, 11) == [1, 3,-49]
    assert coeffBezout(23,26) == [1, -9, 8]
    assert coeffBezout(26, 3) == [1, -1, 9]
    print("coeffBezout : OK")
except:
    print("coeffBezout : ERREUR")
    

coeffBezout : OK


In [7]:
def inverse(a, n) : 
    bezout = coeffBezout(a, n)
    if bezout[0] != 1 : 
        return - 1
    else : 
        return bezout[1] % n

        
try:
    assert inverse(11, 180) == 131
    assert inverse(3, 26) == 9
    assert inverse(23, 26) == 17
    print("inverse : OK")
except:
    print("inverse : ERREUR")
    
    

inverse : OK


Nous avons maintenant toutes les cartes en main pour définir la fonction de déchiffrement affine. Pour rappel : 

\begin{align*}
  D_k \colon \mathbb{Z}/26\mathbb{Z} &\to \mathbb{Z}/26\mathbb{Z}\\
  c_i &\mapsto m_i = \alpha c_i + \beta
\end{align*}

avec $\alpha = a^{-1}$ et $\beta = (-a^{-1}b) \mod n$

où $a^{-1}$ désigne l'inverse modulaire de $a$.



> __Question 4 (déchiffrement)__ : Définir une fonction `dechiffrementAffine(msgChiffre, clef, alphabet)` qui étant donné un message chiffré `msgChiffre`, la `clef` qui a servi à construire ce message chiffré composée de a et b et un alphabet `alphabet` (par défaut, l'alphabet français) renvoie le message en clair correspondant.
> 
> __Contrainte : Pour tout chiffrement symétrique, il est possible d'appeler la fonction de chiffrement pour définir la fonction de déchiffrement. Faites-le ici.__

In [8]:
def dechiffrementAffine(msgChiffre, clef, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    MsgClair = ""
    for letter in msgChiffre :
        letter = lettreToEntier(letter)
        alpha = inverse(clef[0], len(alphabet))
        beta = (-alpha*clef[1]) % len(alphabet)
        lettreEnClair = (alpha*letter + beta) % len(alphabet)
        lettreEnClair = entierToLettre(lettreEnClair)
        MsgClair += lettreEnClair
    return MsgClair
        

try:
    assert dechiffrementAffine("oqggewqqrklecd",[3, 4]) == "messageenclair"
    assert dechiffrementAffine("wueegouutazgih",[23,6]) == "messageenclair"
    print("dechiffrementAffine : OK")
except:
    print("dechiffrementAffine : ERREUR")

dechiffrementAffine : OK


### b) Attaque : Analyse de fréquence

Nous allons maintenant coder une attaque par analyse de fréquence dans le cas du chiffrement affine. Dans le chiffrement par décalage, il n'y avait qu'une seule inconnue : le pas de décalage $k$. Dans le chiffrement affine, il s'agit de trouver les valeurs de $a$ et $b$, ou plus exactement, de $\alpha$ et $\beta$. Il nous faut donc résoudre deux équations à deux inconnues : 

$$
\left\{
    \begin{array}{ll}
        x_c\alpha + \beta & = x_m \mod 26 \\
        y_c\alpha + \beta & = y_m \mod 26 \\
    \end{array}
\right.
$$

où $x_c$ (resp. $y_c$) désigne la lettre qui code $x_m$ (resp. $y_m$) dans le message chiffré.

_Note_ : L'attaque par force brute est toujours très faisable. Il n'y aurait que $Card((\mathbb{Z}/n\mathbb{Z})^*)*Card(\mathbb{Z}/n\mathbb{Z})$ clefs possibles donc pour un alphabet de 26 lettres, cela donne $12 * 26 = 312$ clefs à tester.

__Lisez bien le document "Attaque du chiffrement affine", disponible sur Moodle avant de commencer cette partie !__

> __Question 5 (Résolution de deux équations à deux inconnues.)__ :
> Faire une fonction `resEquations(xc,xm,yc,ym,alphabet)` qui renvoie les valeurs de $\alpha$ et $\beta$ par résolution du système d'équation donné ci-dessus en utilisant le document "Attaque du chiffrement affine" de Moodle.


In [ ]:
import math
"""
resolution des equations
xc * alpha + beta = xm mod 26
yc * alpha + beta = ym mod 26
"""
def resEquations(xc,xm,yc,ym, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    xAlph = lettreToEntier(xm)
    yAlph = lettreToEntier(ym)
    xc = lettreToEntier(xc)
    yc = lettreToEntier(yc)
    d = math.gcd(xAlph-yAlph, xc-yc)
    zAlph = (xAlph-yAlph)//d
    zc = (xc-yc)//d
    zc_inv = inverse(zc, len(alphabet))
    alpha = zAlph*zc_inv % len(alphabet)
    beta = (yAlph - yc * alpha) % len(alphabet)
    retour = (alpha, beta)
    return retour 


try:
    assert resEquations('a','d','d','e') == (9,3)
    assert resEquations('t','e','m','t') == (9,15)
    assert resEquations('t','e','m','a') == (8,8)
    assert resEquations('r','e','f','a') == (9,7)
    print("resEquations : OK")
except:
    print("resEquations : ERREUR")


resEquations : OK


> __Question 6 (attaque par analyse de fréquence)__ :
> Faire une fonction `attaqueFrequenceAff(msgchiffre, frequenceLangue, alphabet)` qui utilise les fonctions précédentes pour décrypter le message en paramètre en comparant les lettres les plus fréquentes dans le message chiffré et dans la langue considérée. `frequenceLangue` est un dictionnaire des fréquences dans la langue du message (le dictionnaire des fréquences de chaque lettre en français `frequenceFrancais` vous est fourni). Il faudra certainement tester plusieurs possibilités...

In [ ]:
frequenceFrancais = {'a': 8.15, 'b': 0.97, 'c': 3.15, 'd': 3.73, 'e': 17.39, 
                     'f': 1.12, 'g': 0.97, 'h': 0.85, 'i': 7.31, 'j': 0.45, 
                     'k': 0.02, 'l': 5.69, 'm': 2.87, 'n': 7.12, 'o': 5.28, 
                     'p': 2.80, 'q': 1.21, 'r': 6.64, 's': 8.14, 't': 7.22, 
                     'u': 6.38, 'v': 1.64, 'w': 0.03, 'x': 0.41, 'y': 0.28, 
                     'z': 0.15}





def attaqueFrequenceAff(msgchiffre, frequenceLangue = frequenceFrancais, alphabet = "abcdefghijklmnopqrstuvwxyz"):
      
      freqMsg = {}
      for letter in alphabet :
         freqMsg[letter] = 0
      for letter in msgchiffre :
         freqMsg[letter] += 1
      for letter in alphabet :
         freqMsg[letter] = freqMsg[letter] / len(msgchiffre) * 100
   
      # Tri des lettres par frequence
      freqLangueSorted = sorted(frequenceLangue.items(), key=lambda x: x[1], reverse=True)
      freqMsgSorted = sorted(freqMsg.items(), key=lambda x: x[1], reverse=True)
   
      # les 5 lettres les plus frequentes
      lettresLangue = [freqLangueSorted[i][0] for i in range(5)]
      lettresMsg = [freqMsgSorted[i][0] for i in range(5)]
   
      # Essai de toutes les combinaisons possibles
      for i in range(5):
         for j in range(5):
               if i != j :
                  clef = resEquations(lettresMsg[0], lettresLangue[i], lettresMsg[1], lettresLangue[j], alphabet)
                  msgDechiffre = dechiffrementAffine(msgchiffre, clef, alphabet)
                  print(f"Essai avec {lettresMsg[0]}->{lettresLangue[i]} et {lettresMsg[1]}->{lettresLangue[j]} : clef={clef} -> {msgDechiffre}")

c1 = "svnhfmmvshpfdskrsfsklvorensrfkkfbnryfefsfmzhroruerbnrslrofshmrlfhonladuuerprskfuudsrfkkrskdvsdmufnkmfdhhrnsyrnorgrnmfyerpdrermrkkeronladuuersrlveerhyvsoyfhuvelrprskfmfyerpdrermrkkeronuefslfdhdorpyvnemfornwdrpr"
print(attaqueFrequenceAff(c1))

print("\nUn peu plus complique... Essayer d'autres choses ?")
c2 = "ntjmpumgxpqtstgqpgtxpnchumtputgfsftgthnngxnchumwxootrtumhpyctgktjqtjchfooxujqhgztumxpotjxotfoqtohrxumhzutwftgtopfmntjmpuatmfmshodpfrxpjjtqtghbxuj"



Essai avec r->e et f->a : clef=(9, 7) -> hqsauppqhayuohjehuhjmqversheujjuisezuruhupcaevenreisehmevuhapemuavsmfonnreyehjunnoheujjehjoqhopnusjpuoaaeshzesvexespuzreyoerepejjrevsmfonnrehemqrreazqhvzuanqrmeyehjupuzreyoerepejjrevsnruhmuoaoveyzqsrpuvestoeye
Essai avec r->e et f->s : clef=(7, 9) -> fyiwsttyfwmsofpqfsfpeyxqdifqsppskiqrsdsfstgwqxqjdqkiqfeqxsfwtqeswxievojjdqmqfpsjjofqsppqfpoyfotjsiptsowwqifrqixqhqitsrdqmoqdqtqppdqxievojjdqfqeyddqwryfxrswjydeqmqfpstsrdqmoqdqtqppdqxijdsfesowoxqmryidtsxqinoqmq
Essai avec r->e et f->i : clef=(17, 1) -> bsqiottsbikoubzebobzwsnerqbeozzoaqejorobotgienevreaqebwenobitewoinqwduvvrekebzovvubeozzebzusbutvoqztouiieqbjeqneleqtojrekueretezzrenqwduvvrebewsrreijsbnjoivsrwekebzotojrekueretezzrenqvrobwouiunekjsqrtoneqpueke
Essai avec r->e et f->t : clef=(5, 20) -> kvjnxoovknzxhkypkxkytvepcjkpxyyxrjpgxcxkxobnpepacprjpktpexknoptxnejtwhaacpzpkyxaahkpxyypkyhvkhoaxjyoxhnnpjkgpjepspjoxgcpzhpcpopyycpejtwhaacpkptvccpngvkegxnavctpzpkyxoxgcpzhpcpopyycpejacxktxhnhepzgvjcoxepj